In [1]:
import os
IN_KAGGLE = 'KAGGLE_URL_BASE' in os.environ
IN_KAGGLE

False

In [2]:
if IN_KAGGLE:
    from kaggle_secrets import UserSecretsClient
    user_secrets = UserSecretsClient()
    secret_value = user_secrets.get_secret("gittoken2")

    !git clone https://{secret_value}@github.com/moienr/TemporalGAN.git

In [3]:
if IN_KAGGLE:
    import time
    import os
    sleep_time = 5
    while not os.path.exists("/kaggle/working/TemporalGAN"):
        print("didn't find the path, wating {sleep_time} more seconds...")
        time.sleep(sleep_time)
    print("path found...")
    import sys
    sys.path.append("/kaggle/working/TemporalGAN")

In [4]:
import torch
torch.__version__

c:\Users\moi3n\miniconda3\envs\pytorchGPU\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'1.13.1'

In [5]:
import numpy as np
from torch.utils.data import Dataset
from glob import glob
from skimage import io
import os
from torchvision import datasets, transforms
import matplotlib
import os
import gc
import random
from datetime import date, datetime
import json
import pprint
os.cpu_count()

20

In [6]:
# create a folder called 'results' in the current directory if it doesn't exist
if not os.path.exists('results'):
    os.mkdir('results')

In [7]:
# Format the date and time
now = datetime.now()
start_string = now.strftime("%Y-%m-%d %H:%M:%S")
print("Current Date and Time:", start_string)

Current Date and Time: 2023-05-07 23:17:49


In [8]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DEVICE

'cuda'

In [9]:
from dataset.data_loaders import *
from dataset.utils.plot_utils import plot_s1s2_tensors, save_s1s2_tensors_plot
#from config import *
from train_utils import *

In [10]:
from temporalgan.gen_v3 import Generator as GeneratorV3
from temporalgan.gen_v2_1 import Generator as GeneratorV2_1
from temporalgan.gen_v2_2 import Generator as GeneratorV2_2
from temporalgan.gen_v2_3 import Generator as GeneratorV2_3
from temporalgan.gen_v1_1 import Generator as GeneratorV1_1
from temporalgan.gen_v1_2 import Generator as GeneratorV1_2
from temporalgan.gen_v1_3 import Generator as GeneratorV1_3
from temporalgan.gen_v1_4 import Generator as GeneratorV1_4
from temporalgan.gen_v1_5 import Generator as GeneratorV1_5
from temporalgan.gen_v1_6 import Generator as GeneratorV1_6
from temporalgan.gen_v1_7 import Generator as GeneratorV1_7
from temporalgan.disc_v2 import Discriminator as DiscriminatorV2
from temporalgan.disc_v1 import Discriminator as DiscriminatorV1
from eval_metrics.loss_function import WeightedL1Loss, reverse_map

In [11]:
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

In [12]:
if IN_KAGGLE:
    s1_t1_dir_train= "/kaggle/input/s1s2-2021-v2/2021/s1_imgs/train"
    s2_t1_dir_train= "/kaggle/input/s1s2-2021-v2/2021/s2_imgs/train"
    s1_t2_dir_train= "/kaggle/input/s1s2-2019-v2/2019/s1_imgs/train"
    s2_t2_dir_train= "/kaggle/input/s1s2-2019-v2/2019/s2_imgs/train"
    s1_t1_dir_test = "/kaggle/input/s1s2-2021-v2/2021/s1_imgs/test"
    s2_t1_dir_test = "/kaggle/input/s1s2-2021-v2/2021/s2_imgs/test"
    s1_t2_dir_test = "/kaggle/input/s1s2-2019-v2/2019/s1_imgs/test"
    s2_t2_dir_test = "/kaggle/input/s1s2-2019-v2/2019/s2_imgs/test"
else:
    s1_t1_dir_train="E:\\s1s2\\s1s2_patched_light\\s1s2_patched_extra_light\\2021\\s1_imgs\\test"
    s2_t1_dir_train="E:\\s1s2\\s1s2_patched_light\\s1s2_patched_extra_light\\2021\\s2_imgs\\test"
    s1_t2_dir_train="E:\\s1s2\\s1s2_patched_light\\s1s2_patched_extra_light\\2019\\s1_imgs\\test"
    s2_t2_dir_train="E:\\s1s2\\s1s2_patched_light\\s1s2_patched_extra_light\\2019\\s2_imgs\\test"
    s1_t1_dir_test="E:\\s1s2\\s1s2_patched_light\\s1s2_patched_extra_light\\2021\\s1_imgs\\test"
    s2_t1_dir_test="E:\\s1s2\\s1s2_patched_light\\s1s2_patched_extra_light\\2021\\s2_imgs\\test"
    s1_t2_dir_test="E:\\s1s2\\s1s2_patched_light\\s1s2_patched_extra_light\\2019\\s1_imgs\\test"
    s2_t2_dir_test="E:\\s1s2\\s1s2_patched_light\\s1s2_patched_extra_light\\2019\\s2_imgs\\test"

In [13]:
TWO_WAY_DATASET = True if IN_KAGGLE else False
INPUT_CHANGE_MAP = True if IN_KAGGLE else False
S2_INCHANNELS = 12 if INPUT_CHANGE_MAP else 6
S1_INCHANNELS = 7 if INPUT_CHANGE_MAP else 1

LEARNING_RATE = 2e-4
BATCH_SIZE = 4 if IN_KAGGLE else 1
NUM_WORKERS = 2 if IN_KAGGLE else 8
IMAGE_SIZE = 256
WEIGHTED_LOSS = True
L1_LAMBDA = 100
CHANGED_L1_WEIGHT = 5
NUM_EPOCHS = 10 if IN_KAGGLE else 1

LOAD_MODEL = False
SAVE_MODEL = True if IN_KAGGLE else False
SAVE_MODEL_EVERY_EPOCH = 10
SAVE_EXAMPLE_PLOTS = True
EXAMPLES_TO_PLOT = [1,32,64,128,256,512,1024] if IN_KAGGLE else [1,2]
CHECKPOINT_DISC = "disc.pth.tar"
CHECKPOINT_GEN = "gen.pth.tar"

RANDOM_SEED = 75

In [14]:
def main():
    disc = DiscriminatorV1(s2_in_channels=S2_INCHANNELS, s1_in_channels=S1_INCHANNELS).to(DEVICE)
#     gen = GeneratorV2(s2_in_channels=S2_INCHANNELS, s1_in_channels= S1_INCHANNELS, features=64,pam_downsample=2).to(DEVICE)
    gen = GeneratorV1_3(s2_in_channels=S2_INCHANNELS, s1_in_channels= S1_INCHANNELS, features=64).to(DEVICE)
#     gen = GeneratorV2_3(s2_in_channels=S2_INCHANNELS, s1_in_channels= S1_INCHANNELS, features=64).to(DEVICE)
    opt_disc = optim.Adam(disc.parameters(), lr=LEARNING_RATE, betas=(0.5, 0.999),)
    opt_gen = optim.Adam(gen.parameters(), lr=LEARNING_RATE, betas=(0.5, 0.999))
    BCE = nn.BCEWithLogitsLoss()
    if WEIGHTED_LOSS:
        L1_LOSS = WeightedL1Loss(change_weight=CHANGED_L1_WEIGHT)
    else:
        L1_LOSS = nn.L1Loss()

    if LOAD_MODEL:
        load_checkpoint(
            CHECKPOINT_GEN, gen, opt_gen, LEARNING_RATE,
        )
        load_checkpoint(
            CHECKPOINT_DISC, disc, opt_disc, LEARNING_RATE,
        )

    transform = transforms.Compose([S2S1Normalize(),myToTensor(dtype=torch.float32)])


    train_dataset = Sen12Dataset(s1_t1_dir=s1_t1_dir_train,
                                 s2_t1_dir=s2_t1_dir_train,
                                 s1_t2_dir=s1_t2_dir_train,
                                 s2_t2_dir=s2_t2_dir_train,
                                 transform=transform,
                                 two_way=TWO_WAY_DATASET)
    
    train_loader = DataLoader(
        train_dataset,
        batch_size=BATCH_SIZE,
        shuffle=True,
        num_workers=NUM_WORKERS,
    )
    g_scaler = torch.cuda.amp.GradScaler()
    d_scaler = torch.cuda.amp.GradScaler()
#     val_dataset = MapDataset(root_dir=VAL_DIR)
#     val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False)

    for epoch in range(1, NUM_EPOCHS+1):
        print("\n\n" , end="")
        print(f"Epoch: {epoch}")
        print("")
        train_fn(
            disc, gen, train_loader, opt_disc, opt_gen,
            L1_LOSS, BCE, g_scaler, d_scaler, weighted_loss= WEIGHTED_LOSS,
            cm_input=INPUT_CHANGE_MAP, grad_clip=False)

        if SAVE_MODEL and epoch % SAVE_MODEL_EVERY_EPOCH == 0:
            save_checkpoint(epoch,gen, opt_gen, filename=CHECKPOINT_GEN)
            save_checkpoint(epoch,disc, opt_disc, filename=CHECKPOINT_DISC)


        if SAVE_EXAMPLE_PLOTS:
            for img_i in EXAMPLES_TO_PLOT:
                save_some_examples(gen, train_dataset, epoch, folder="train_evaluation_plots",cm_input=INPUT_CHANGE_MAP, img_indx=img_i)

            
        gc.collect()
        torch.cuda.empty_cache()
            
    return gen


In [15]:
torch.manual_seed(RANDOM_SEED)
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

In [16]:
#matplotlib.use('Agg') # This refrains matplot lib form showing the plotted resualts below the cell
gen_model = main()



Epoch: 1



100%|██████████| 4/4 [00:17<00:00,  4.44s/it, D_fake_mean=0.501, D_real_mean=0.497, G_loss_mean=24.5, L1_mean=23.5]


In [17]:
# transform = transforms.Compose([S2S1Normalize(),myToTensor(dtype=torch.float32)])
# train_dataset = Sen12Dataset(s1_t1_dir=s1_t1_dir_train,
#                              s2_t1_dir=s2_t1_dir_train,
#                              s1_t2_dir=s1_t2_dir_train,
#                              s2_t2_dir=s2_t2_dir_train,
#                              transform=transform,
#                              two_way=TWO_WAY_DATASET)
# #plt.imshow(train_dataset[0][-1][0])

# plt.imshow(torch.permute(train_dataset[123][4],(1,2,0))[:,:,0])
# plt.colorbar()
# plt.show()

In [18]:
transform = transforms.Compose([S2S1Normalize(),myToTensor()])
test_dataset = Sen12Dataset(s1_t1_dir=s1_t1_dir_test,
                            s2_t1_dir=s2_t1_dir_test,
                            s1_t2_dir=s1_t2_dir_test,
                            s2_t2_dir=s2_t2_dir_test,
                            transform=transform,
                            two_way=TWO_WAY_DATASET)
test_loader = DataLoader(
        test_dataset,
        batch_size=BATCH_SIZE,
        shuffle=False,
        num_workers=NUM_WORKERS,
    )

In [19]:
# save_some_examples(gen_model, test_dataset, NUM_EPOCHS, folder="evaluation",cm_input=INPUT_CHANGE_MAP, img_indx=2)

In [20]:
def eval_fn(gen, loader, ssim, psnr):
    gen.eval()
    loop = tqdm(loader, leave=True)
    weighted_ssim_list = []
    normal_ssim_list = []
    weighted_psnr_list = []
    normal_psnr_list = []
    reverse_weighted_ssim_list = []
    reverse_weighted_psnr_list = []
    
    for idx, (s2t2,s1t2,s2t1,s1t1,cm,rcm,s1cm) in enumerate(loop):
        s2t2,s1t2,s2t1,s1t1,cm,rcm,s1cm = s2t2.to(DEVICE),s1t2.to(DEVICE),s2t1.to(DEVICE),s1t1.to(DEVICE),cm.to(DEVICE),rcm.to(DEVICE),s1cm.to(DEVICE)
        s2t2,s1t2,s2t1,s1t1,cm,rcm,s1cm = s2t2.to(torch.float32),s1t2.to(torch.float32),s2t1.to(torch.float32),s1t1.to(torch.float32),cm.to(torch.float32),rcm.to(torch.float32),s1cm.to(torch.float32)
        if INPUT_CHANGE_MAP:
            s2t2 = torch.cat((s2t2, cm), dim=1)
            s1t1 = torch.cat((s1t1, rcm), dim=1)
        preds = gen(s2t2, s1t1)
        
        s1cm_reversed = reverse_map(s1cm)
        
        # Rescaling from -1-1 to 0-1
        s1t2 = (s1t2 + 1) / 2
        preds = (preds + 1) / 2
        
        weighted_ssim = ssim((s1t2, preds), s1cm)
        weighted_ssim_list.append(weighted_ssim)
        reverse_weighted_ssim = ssim((s1t2, preds), s1cm_reversed)
        reverse_weighted_ssim_list.append(reverse_weighted_ssim)
        normal_ssim = ssim((s1t2, preds))
        normal_ssim_list.append(normal_ssim)
        
        weighted_psnr = psnr((s1t2, preds), s1cm)
        weighted_psnr_list.append(weighted_psnr)
        reverse_weighted_psnr = psnr((s1t2, preds), s1cm_reversed)
        reverse_weighted_psnr_list.append(reverse_weighted_psnr)
        normal_psnr = psnr((s1t2, preds))
        normal_psnr_list.append(normal_psnr)


        if idx % 10 == 0:
            loop.set_postfix(
                wssim_mean = torch.tensor(weighted_ssim_list).mean().item(),
                ssim_mean = torch.tensor(normal_ssim_list).mean().item(),
                rwssim_mean = torch.tensor(reverse_weighted_ssim_list).mean().item(),
                wpsnr_mean = torch.tensor(weighted_psnr_list).mean().item(),
                psnr_mean = torch.tensor(normal_psnr_list).mean().item(),
                rwpsnr_mean = torch.tensor(reverse_weighted_psnr_list).mean().item(),
                
            )

    
    eval_dict = {"SSIM":{}, "PSNR":{}}
    
    eval_dict["SSIM"]["cwssim_mean"] = torch.tensor(weighted_ssim_list).mean().item()
    eval_dict["SSIM"]["ssim_mean"] = torch.tensor(normal_ssim_list).mean().item()
    eval_dict["SSIM"]["rcwssim_mean"] = torch.tensor(reverse_weighted_ssim_list).mean().item()
    eval_dict["PSNR"]["cwpsnr_mean"] = torch.tensor(weighted_psnr_list).mean().item()
    eval_dict["PSNR"]["psnr_mean"] = torch.tensor(normal_psnr_list).mean().item()
    eval_dict["PSNR"]["rcwpsnr_mean"] = torch.tensor(reverse_weighted_psnr_list).mean().item()
    
    return eval_dict
    
    

In [21]:
from eval_metrics import ssim
from eval_metrics.psnr import wpsnr
wssim = ssim.WSSIM(data_range=1.0)

In [22]:
soft_eval_results = eval_fn(gen_model, test_loader, wssim, wpsnr)
pprint.pprint(soft_eval_results)

 25%|██▌       | 1/4 [00:02<00:08,  2.96s/it, psnr_mean=10.9, rwpsnr_mean=10.9, rwssim_mean=0.0419, ssim_mean=0.0417, wpsnr_mean=10.3, wssim_mean=0.0374]

S1T2 Range:  tensor(-1., device='cuda:0') tensor(1., device='cuda:0')
S1T2 Pred Range:  tensor(-0.9057, device='cuda:0', grad_fn=<MinBackward1>) tensor(0.9079, device='cuda:0', grad_fn=<MaxBackward1>)
S1T2 Range:  tensor(-1., device='cuda:0') tensor(1., device='cuda:0')
S1T2 Pred Range:  tensor(-0.7740, device='cuda:0', grad_fn=<MinBackward1>) tensor(0.7030, device='cuda:0', grad_fn=<MaxBackward1>)
S1T2 Range:  

 75%|███████▌  | 3/4 [00:03<00:00,  1.22it/s, psnr_mean=10.9, rwpsnr_mean=10.9, rwssim_mean=0.0419, ssim_mean=0.0417, wpsnr_mean=10.3, wssim_mean=0.0374]

tensor(-1., device='cuda:0') tensor(1., device='cuda:0')
S1T2 Pred Range:  tensor(-0.8609, device='cuda:0', grad_fn=<MinBackward1>) tensor(0.8613, device='cuda:0', grad_fn=<MaxBackward1>)
S1T2 Range:  tensor(-1., device='cuda:0') tensor(1., device='cuda:0')
S1T2 Pred Range:  tensor(-0.8481, device='cuda:0', grad_fn=<MinBackward1>) tensor(0.8292, device='cuda:0', grad_fn=<MaxBackward1>)


100%|██████████| 4/4 [00:03<00:00,  1.09it/s, psnr_mean=10.9, rwpsnr_mean=10.9, rwssim_mean=0.0419, ssim_mean=0.0417, wpsnr_mean=10.3, wssim_mean=0.0374]

{'PSNR': {'cwpsnr_mean': 10.312030792236328,
          'psnr_mean': 10.557164192199707,
          'rcwpsnr_mean': 10.566265106201172},
 'SSIM': {'cwssim_mean': 0.03583900257945061,
          'rcwssim_mean': 0.038773730397224426,
          'ssim_mean': 0.0386761799454689}}


In [3]:
from changedetection.utils import get_column_values
hard_test_names = get_column_values("./changedetection/chaned_pairs.csv", "name")
len(hard_test_names)

148

In [4]:
transform = transforms.Compose([S2S1Normalize(),myToTensor()])
hard_test_dataset = Sen12DatasetHardTest(s1_t1_dir=s1_t1_dir_test,
                            s2_t1_dir=s2_t1_dir_test,
                            s1_t2_dir=s1_t2_dir_test,
                            s2_t2_dir=s2_t2_dir_test,
                            hard_test_names=hard_test_names,
                            transform=transform,
                            two_way=TWO_WAY_DATASET)
hard_test_dataset = DataLoader(
        test_dataset,
        batch_size=BATCH_SIZE,
        shuffle=False,
        num_workers=NUM_WORKERS,
    )

NameError: name 'transforms' is not defined

In [ ]:
hard_eval_results = eval_fn(gen_model, test_loader, wssim, wpsnr)
pprint.pprint(hard_eval_results)

In [23]:
# Format the date and time
now = datetime.now()
finish_string = now.strftime("%Y-%m-%d %H:%M:%S")
file_name = now.strftime("D_%Y_%m_%d_T_%H_%M")
print("Current Date and Time:", finish_string)
print("File Name:", file_name)

Current Date and Time: 2023-05-07 23:18:14
File Name: D_2023_05_07_T_23_18


In [24]:
log_json = {}
log_json["Time"] = {"Start": start_string, "Finish": finish_string}
log_json["IN_KAGGLE"] = IN_KAGGLE
log_json["TWO_WAY_DATASET"] = TWO_WAY_DATASET
log_json["INPUT_CHANGE_MAP"] = INPUT_CHANGE_MAP
log_json["S2_INCHANNELS"] = S2_INCHANNELS
log_json["S1_INCHANNELS"] = S1_INCHANNELS
log_json["LEARNING_RATE"] = LEARNING_RATE
log_json["BATCH_SIZE"] = BATCH_SIZE
log_json["NUM_WORKERS"] = NUM_WORKERS
log_json["IMAGE_SIZE"] = IMAGE_SIZE
log_json["WEIGHTED_LOSS"] = WEIGHTED_LOSS
log_json["L1_LAMBDA"] = L1_LAMBDA
log_json["CHANGED_L1_WEIGHT"] = CHANGED_L1_WEIGHT
log_json["NUM_EPOCHS"] = NUM_EPOCHS
log_json["LOAD_MODEL"] = LOAD_MODEL
log_json["SAVE_MODEL"] = SAVE_MODEL
log_json["SAVE_MODEL_EVERY_EPOCH"] = SAVE_MODEL_EVERY_EPOCH
log_json["SAVE_EXAMPLE_PLOTS"] = SAVE_EXAMPLE_PLOTS
log_json["EXAMPLES_TO_PLOT"] = EXAMPLES_TO_PLOT
log_json["CHECKPOINT_DISC"] = CHECKPOINT_DISC
log_json["CHECKPOINT_GEN"] = CHECKPOINT_GEN
log_json["RANDOM_SEED"] = RANDOM_SEED
log_json["SoftEval"] = {"Soft Results": soft_eval_results}
log_json["HardEval"] = {"Hard Results": hard_eval_results}

with open(f"results/RUN_{file_name}.json", "w") as fp:
    json.dump(log_json, fp, indent=4)

In [25]:
# import shutil
# shutil.rmtree("test_evaluation_plots")

In [26]:
for img_i in range(1,len(test_dataset)//2, 2):
    save_some_examples(gen_model, test_dataset, NUM_EPOCHS, folder="test_evaluation_plots_1",cm_input=INPUT_CHANGE_MAP, img_indx=img_i)
!zip -r /kaggle/working/test_evalplot1.zip  /kaggle/working/test_evaluation_plots_1/

'zip' is not recognized as an internal or external command,
operable program or batch file.


In [27]:
for img_i in range((len(test_dataset)//2)+1,len(test_dataset), 2):
    save_some_examples(gen_model, test_dataset, NUM_EPOCHS, folder="test_evaluation_plots_2",cm_input=INPUT_CHANGE_MAP, img_indx=img_i)
!zip -r /kaggle/working/test_evalplot2.zip  /kaggle/working/test_evaluation_plots_2/

'zip' is not recognized as an internal or external command,
operable program or batch file.


In [28]:
# !zip -r /kaggle/working/train_evalplot.zip  /kaggle/working/train_evaluation_plots/

In [29]:
# from IPython.display import FileLink
# FileLink(r'/kaggle/working/evalplot.zip')

In [30]:
# def eval_graph_fn(gen, loader, ssim, psnr):
#     gen.eval()
#     loop = tqdm(loader, leave=True)
    
#     wssim_ratio_list = []
#     wpsnr_ratio_list = []
#     s1cm_mean_list   = []
    
#     for idx, (s2t2,s1t2,s2t1,s1t1,cm,rcm,s1cm) in enumerate(loop):
#         s2t2,s1t2,s2t1,s1t1,cm,rcm,s1cm = s2t2.to(DEVICE),s1t2.to(DEVICE),s2t1.to(DEVICE),s1t1.to(DEVICE),cm.to(DEVICE),rcm.to(DEVICE),s1cm.to(DEVICE)
#         s2t2,s1t2,s2t1,s1t1,cm,rcm,s1cm = s2t2.to(torch.float32),s1t2.to(torch.float32),s2t1.to(torch.float32),s1t1.to(torch.float32),cm.to(torch.float32),rcm.to(torch.float32),s1cm.to(torch.float32)
#         if INPUT_CHANGE_MAP:
#             s2t2 = torch.cat((s2t2, cm), dim=1)
#             s1t1 = torch.cat((s1t1, rcm), dim=1)
#         preds = gen(s2t2, s1t1)
        
#         s1t2_list = s1t2.to("cpu").chunk(s1t2.size(0))
#         preds_list = preds.to("cpu").chunk(preds.size(0))
#         s1cm_list = s1cm.to("cpu").chunk(s1cm.size(0))
        
        
#         wssim_ratio_batch_list = [ssim((s1t2, preds), s1cm)/ssim((s1t2, preds)) for s1t2, preds, s1cm in zip(s1t2_list, preds_list, s1cm_list)]
#         wpsnr_ratio_batch_list = [psnr((s1t2, preds))-psnr((s1t2, preds), s1cm) for s1t2, preds, s1cm in zip(s1t2_list, preds_list, s1cm_list)]
#         s1cm_mean_batch_list   = [s1cm.mean() for s1cm in s1cm_list]
        
#         wssim_ratio_list = wssim_ratio_list + wssim_ratio_batch_list
#         wpsnr_ratio_list = wpsnr_ratio_list + wpsnr_ratio_batch_list
#         s1cm_mean_list   = s1cm_mean_list   + s1cm_mean_batch_list  
    
#     return wssim_ratio_list, wpsnr_ratio_list, s1cm_mean_list

In [31]:
# ssim_ratio_list, wpsnr_ratio_list, s1cm_mean_list = eval_graph_fn(gen_model, test_loader, wssim, wpsnr)

In [32]:
# plt.plot(ssim_ratio_list,s1cm_mean_list)